In [88]:
import pandas as pd
import numpy as np
import random 
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, GlobalMaxPooling1D, concatenate, GlobalAveragePooling1D

In [89]:
db = pd.read_csv('NIFTY50_all.csv', header=0)

In [90]:
print(db.head())

x = {}

for i in range(len(db)):
    x[db.Symbol[i]] = 0

         Date      Symbol Series  Prev Close    Open     High    Low   Last  \
0  2007-11-27  MUNDRAPORT     EQ      440.00  770.00  1050.00  770.0  959.0   
1  2007-11-28  MUNDRAPORT     EQ      962.90  984.00   990.00  874.0  885.0   
2  2007-11-29  MUNDRAPORT     EQ      893.90  909.00   914.75  841.0  887.0   
3  2007-11-30  MUNDRAPORT     EQ      884.20  890.00   958.00  890.0  929.0   
4  2007-12-03  MUNDRAPORT     EQ      921.55  939.75   995.00  922.0  980.0   

    Close    VWAP    Volume      Turnover  Trades  Deliverable Volume  \
0  962.90  984.72  27294366  2.687719e+15     NaN           9859619.0   
1  893.90  941.38   4581338  4.312765e+14     NaN           1453278.0   
2  884.20  888.09   5124121  4.550658e+14     NaN           1069678.0   
3  921.55  929.17   4609762  4.283257e+14     NaN           1260913.0   
4  969.30  965.65   2977470  2.875200e+14     NaN            816123.0   

   %Deliverble  
0       0.3612  
1       0.3172  
2       0.2088  
3       0.2735  
4

In [91]:
min_return = min(db[['Close']].min(axis=0))
max_return = max(db[['Close']].max(axis=0))

db['Close'] = (db['Close'] - min_return) / (max_return - min_return)

In [92]:
print(len(x))

#print(db.Date[:20])
#print(db.Symbol[:20])

sym = db.Symbol[0]

temp = [[]]*65
#print(temp)
count = 0

for i in range(len(db)):
    if db.Symbol[i] == sym:
        temp[count].append(db.Close[i])
    else:
        count += 1
        sym = db.Symbol[i]

65


In [93]:
N = 10
n_rows = 10000
X = []
Y = []

In [94]:
for i in range(65):
    for k in range(n_rows//65):
        j = random.randint(0, len(temp[i])-N)
        X.append(temp[i][j:j+N])
        Y.append(temp[i][j+N])

In [95]:
len(X)

9945

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [97]:
def create_model():
    in_seq = Input(shape = (N, 1))

    x = Bidirectional(LSTM(128, return_sequences=True))(in_seq)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x) 
            
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    out = Dense(1, activation="linear")(conc)      

    model = Model(inputs=in_seq, outputs=out)
    model.compile(loss="mse", optimizer="adam", metrics=['mae', 'mape'])    
    return model

In [98]:
model = create_model()
model.fit(X_train, y_train, batch_size=2048, epochs=200)

Epoch 1/200
4/4 [==============================] - 10s 671ms/step - loss: 0.0060 - mae: 0.0334 - mape: 213.8705
Epoch 2/200
4/4 [==============================] - 3s 655ms/step - loss: 9.0429e-04 - mae: 0.0235 - mape: 380.3529
Epoch 3/200
4/4 [==============================] - 3s 660ms/step - loss: 7.6561e-04 - mae: 0.0211 - mape: 133.1764
Epoch 4/200
4/4 [==============================] - 3s 731ms/step - loss: 3.3649e-04 - mae: 0.0155 - mape: 310.9960
Epoch 5/200
4/4 [==============================] - 3s 748ms/step - loss: 1.2047e-04 - mae: 0.0085 - mape: 162.3427
Epoch 6/200
4/4 [==============================] - 3s 703ms/step - loss: 1.3916e-04 - mae: 0.0092 - mape: 56.1593
Epoch 7/200
4/4 [==============================] - 3s 717ms/step - loss: 8.5301e-05 - mae: 0.0064 - mape: 120.3095
Epoch 8/200
4/4 [==============================] - 3s 700ms/step - loss: 4.5552e-05 - mae: 0.0032 - mape: 53.4895
Epoch 9/200
4/4 [==============================] - 3s 705ms/step - loss: 4.8726e-05 -

In [99]:
results = model.evaluate(X_test, y_test, batch_size=2048)

2/2 [==============================] - 2s 206ms/step - loss: 3.3420e-05 - mae: 0.0020 - mape: 19.4399


In [100]:
results = model.predict(X_test[1:2])

print(X_test[1], results, y_test[1])

[0.02449258510690109, 0.02552141674377831, 0.025626430623873767, 0.02606779330833293, 0.02641479569473531, 0.026364571665124438, 0.02649545853017095, 0.02662634539521746, 0.02649545853017095, 0.02618954853163201] [[0.02542323]] 0.02584711196610335


In [101]:
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 10, 1)]      0                                            
__________________________________________________________________________________________________
bidirectional_56 (Bidirectional (None, 10, 256)      133120      input_21[0][0]                   
__________________________________________________________________________________________________
bidirectional_57 (Bidirectional (None, 10, 256)      394240      bidirectional_56[0][0]           
__________________________________________________________________________________________________
bidirectional_58 (Bidirectional (None, 10, 128)      164352      bidirectional_57[0][0]           
___________________________________________________________________________________________